In [ ]:
!pip install deeplake

In [3]:
import deeplake
import torch
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F

In [4]:
train_ds = deeplake.load('hub://activeloop/nih-chest-xray-train')
test_ds = deeplake.load('hub://activeloop/nih-chest-xray-test')

train_ds.visualize()

|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/nih-chest-xray-train



|

hub://activeloop/nih-chest-xray-train loaded successfully.



\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/nih-chest-xray-test



\

hub://activeloop/nih-chest-xray-test loaded successfully.



HINT: Please forward the port - 60121 to your local machine, if you are running on the cloud.
 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


In [6]:
print(test_ds.tensors.keys())

dict_keys(['images', 'findings', 'boxes/bbox', 'boxes/finding', 'metadata/patient_id', 'metadata/patient_age', 'metadata/patient_gender', 'metadata/follow_up_num', 'metadata/view_position', 'metadata/orig_img_w', 'metadata/orig_img_h', 'metadata/orig_img_pix_spacing_x', 'metadata/orig_img_pix_spacing_y'])


In [7]:
test_ds['findings']

Tensor(key='findings')

In [112]:
labels = ["No Finding",
          "Hernia",
          "Emphysema",
          "Nodule",
          "Pneumonia",
          "Consolidation",
          "Cardiomegaly",
          "Effusion",
          "Mass",
          "Pleural_Thickening",
          "Atelectasis",
          "Pneumothorax",
          "Fibrosis",
          "Infiltration",
          "Edema"
          ]

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

"""
def findings_transform(findings_list):
    multi_hot_encoded = [0] * len(labels)
    for index in findings_list:
        multi_hot_encoded[index] = 1
    return torch.Tensor(multi_hot_encoded)
"""

def findings_transform(findings_list):

    multi_hot_encoded = [0] * len(labels)
    for index in findings_list:
        multi_hot_encoded[index] = 1
    return torch.Tensor(multi_hot_encoded)

num_workers = 0
batch_size = 128

train_loader = train_ds.pytorch(num_workers=num_workers, batch_size=batch_size, transform={
                        'images': image_transform, 'findings': findings_transform}, shuffle=True, decode_method={'images': 'pil'})

test_loader = test_ds.pytorch(num_workers=num_workers, batch_size=batch_size, transform={
                        'images': image_transform, 'findings': findings_transform}, shuffle=True, decode_method={'images': 'pil'})

In [ ]:
"""
for batch in test_loader:
  print(batch['findings'][0])
  print(batch['findings'][0].shape)
  print(batch['findings'].shape)

  print(batch['images'][0])
  print(batch['images'][0].shape)
  print(batch['images'].shape)
  break
  """

In [97]:
#for batch in train_loader:
  #print(batch['findings'][0])
  #print(batch['findings'][0].shape)
  #print(batch['findings'].shape)

  #print(batch['images'][0])
  #print(batch['images'][0].shape)
  #print(batch['images'].shape)
  #break

torch.Size([128, 15])
torch.Size([128, 15])
torch.Size([128, 15])
torch.Size([128, 15])
torch.Size([128, 15])
torch.Size([128, 15])
torch.Size([128, 15])
torch.Size([128, 15])


KeyboardInterrupt: ignored

In [39]:
"""
num_workers = 0
batch_size = 128

train_loader = train_ds.pytorch(num_workers=num_workers, batch_size=batch_size, transform={
                        'images': image_transform, 'findings': findings_transform}, shuffle=True, decode_method={'images': 'pil'})

test_loader = test_ds.pytorch(num_workers=num_workers, batch_size=batch_size, transform={
                        'images': image_transform, 'findings': findings_transform}, shuffle=True, decode_method={'images': 'pil'})
"""

In [29]:
"""
from torch.utils.data import random_split

train_dataset = train_loader.dataset

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_set, val_set = random_split(train_dataset, [train_size, val_size])

from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=train_loader.batch_size, shuffle=True, num_workers=train_loader.num_workers)
val_loader = DataLoader(val_set, batch_size=train_loader.batch_size, shuffle=True, num_workers=train_loader.num_workers)

print(test_loader)
print(train_loader)
"""

'\nfrom torch.utils.data import random_split\n\ntrain_dataset = train_loader.dataset\n\ntrain_size = int(0.8 * len(train_dataset))\nval_size = len(train_dataset) - train_size\n\ntrain_set, val_set = random_split(train_dataset, [train_size, val_size])\n\nfrom torch.utils.data import DataLoader\n\ntrain_loader = DataLoader(train_set, batch_size=train_loader.batch_size, shuffle=True, num_workers=train_loader.num_workers)\nval_loader = DataLoader(val_set, batch_size=train_loader.batch_size, shuffle=True, num_workers=train_loader.num_workers)\n\nprint(test_loader)\nprint(train_loader)\n'

In [114]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [115]:
class ChestXrayCNN(nn.Module):
    def __init__(self, num_classes=15):
        super(ChestXrayCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.sigmoid(self.fc(x))
        return x

In [126]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ChestXrayCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [138]:
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    inner_loop = 0
    for batch in train_loader:
    
        #inputs = batch['images']
        #labels = batch['findings']
        inner_loop = inner_loop + 1
        print(inner_loop) #664
        optimizer.zero_grad()
        outputs = model(batch['images'])
        #print(batch['images'].shape)
        #print(batch['findings'].shape)

        loss = criterion(outputs, batch['findings'])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


KeyboardInterrupt: ignored

In [120]:
dataloader_test = train_ds.tensorflow()

print(dataloader_test)

<DatasetAdapter element_spec={'findings': TensorSpec(shape=(None,), dtype=tf.uint32, name=None), 'images': TensorSpec(shape=(1024, 1024, None), dtype=tf.uint8, name=None), 'metadata/follow_up_num': TensorSpec(shape=(1,), dtype=tf.int32, name=None), 'metadata/orig_img_h': TensorSpec(shape=(1,), dtype=tf.uint32, name=None), 'metadata/orig_img_pix_spacing_x': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'metadata/orig_img_pix_spacing_y': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'metadata/orig_img_w': TensorSpec(shape=(1,), dtype=tf.uint32, name=None), 'metadata/patient_age': TensorSpec(shape=(1,), dtype=tf.int32, name=None), 'metadata/patient_gender': TensorSpec(shape=(1,), dtype=tf.uint32, name=None), 'metadata/patient_id': TensorSpec(shape=(1,), dtype=tf.int32, name=None), 'metadata/view_position': TensorSpec(shape=(1,), dtype=tf.uint32, name=None)}>


In [121]:
#dataloader_test['Finding']

TypeError: ignored

In [129]:
import deeplake
import torch
from torchvision import transforms, models

ds = deeplake.load('hub://activeloop/cifar10-train')

print(ds)

ds.tensors.keys()    # dict_keys(['images', 'labels'])
ds.labels[0].numpy() # array([6], dtype=uint32)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/cifar10-train



/

hub://activeloop/cifar10-train loaded successfully.



Dataset(path='hub://activeloop/cifar10-train', read_only=True, tensors=['images', 'labels'])


array([6], dtype=uint32)

In [130]:
import deeplake
import torch
from torchvision import transforms, models

ds = deeplake.load('hub://activeloop/nih-chest-xray-train')

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/nih-chest-xray-train



/

hub://activeloop/nih-chest-xray-train loaded successfully.

Dataset(path='hub://activeloop/nih-chest-xray-train', read_only=True, tensors=['findings', 'images', 'metadata/follow_up_num', 'metadata/orig_img_h', 'metadata/orig_img_pix_spacing_x', 'metadata/orig_img_pix_spacing_y', 'metadata/orig_img_w', 'metadata/patient_age', 'metadata/patient_gender', 'metadata/patient_id', 'metadata/view_position'])


In [131]:
print(ds)

Dataset(path='hub://activeloop/nih-chest-xray-train', read_only=True, tensors=['findings', 'images', 'metadata/follow_up_num', 'metadata/orig_img_h', 'metadata/orig_img_pix_spacing_x', 'metadata/orig_img_pix_spacing_y', 'metadata/orig_img_w', 'metadata/patient_age', 'metadata/patient_gender', 'metadata/patient_id', 'metadata/view_position'])


In [136]:
ds.tensors.keys()

dict_keys(['findings', 'images', 'metadata/follow_up_num', 'metadata/orig_img_h', 'metadata/orig_img_pix_spacing_x', 'metadata/orig_img_pix_spacing_y', 'metadata/orig_img_w', 'metadata/patient_age', 'metadata/patient_gender', 'metadata/patient_id', 'metadata/view_position'])